<a href="https://colab.research.google.com/github/Huvinesh-Rajendran-12/Computer-Vision/blob/main/050521_ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Operations

## Import Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import cv2 as cv 
from google.colab.patches import cv2_imshow 
from skimage import io 
from PIL import Image 
import matplotlib.pylab as plt 


In [ ]:
data_dir = "/content/drive/MyDrive/Practical/Computer Vision and NLp/tom-coe-FXhRrfFH9kc-unsplash.jpg"
image = cv.imread(data_dir)
cv2_imshow(image)
cv.waitKey(0)

In [ ]:
# Check the image matrix data type (could know the bit depth of the image)
print(image.dtype)
# Check the height of image 
print(image.shape[0])
# Check the width of image 
print(image.shape[1])
# Check the number of channels of the image
print(image.shape[2])

In [ ]:
print(image.shape)

## Resizing(Scaling) 

In [ ]:
# to prevent image does from skewed or distorted 
# calculate the ratio of the new image to the old image
r = 500 / image.shape[1]
dim = (500, int(image.shape[0] * r))

# perform the actual resizing of the image and show it
resized = cv.resize(image, dim, interpolation = cv.INTER_AREA)
cv2_imshow(resized)
cv.waitKey(0)

## Rotating Image

In [ ]:
# grab the dimensions of the image and calculate the center
(h, w) = resized.shape[:2]
center = (w / 2, h / 2)

# rotate the image by 180 degrees
M = cv.getRotationMatrix2D(center, 180, 1.0)
rotated = cv.warpAffine(resized, M, (w, h))
cv2_imshow(rotated)

# re-rotate
M = cv.getRotationMatrix2D(center, 180, 1.0)
rotated = cv.warpAffine(rotated, M, (w, h))
cv2_imshow(rotated)
cv.waitKey(0)

## Histogram

In [ ]:
plt.hist(rotated.ravel(),bins = 256, range = [0,256]) 
plt.show()

### Display the histogram of RGB channel

In [ ]:
color = ('r','g','b')
for i,col in enumerate(color):
    histr = cv.calcHist([rotated],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()

In [ ]:
gray_image = cv.cvtColor(rotated, cv.COLOR_RGB2GRAY)
cv2_imshow(gray_image)